In [1]:
# Importing all the required packages
import arcpy
from arcpy.ia import *
import os, sys
from arcgis.gis import GIS
from arcgis.learn import export_training_data
from arcgis.learn import prepare_data
from arcgis.learn import YOLOv3
from arcgis.learn import FasterRCNN
from datetime import datetime
# Check perform to see if Image Analyst is available
# arcpy.CheckOutExtension("ImageAnalyst")

In [3]:
# Setting processor type and parallel processing factor
arcpy.env.processorType = "CPU"
# Parallel Processing
arcpy.env.parallelProcessingFactor = "50%"

In [21]:
RGB_imagery = r"E:\Nitin.Rai\ASABE2021\Dataset\sunflower_Clip_2.tif"
arcpy.env.workspace = RGB_imagery
bands = [Raster(os.path.join(RGB_imagery, b))
         for b in arcpy.ListRasters()]

In [23]:
path = os.path.dirname(RGB_imagery)
base1 = os.path.basename(RGB_imagery)
base = os.path.splitext(base1)[0]
ExGreen_filename = base + "_ExcessGreen4.tif"
total_bands = (Float(bands[0] + bands[1] + bands[2]))
red_band = (Float(bands[0]) / total_bands)
green_band = (Float(bands[1]) / total_bands)
blue_band = (Float(bands[2]) / total_bands)
ExGreen = (2 * green_band - red_band - blue_band)
ExGreen.save(ExGreen_filename)
arcpy.env.workspace = path
print ("You have successfully exported the Excess Green.tif file")
arcpy.BuildPyramids_management(ExGreen_filename, "", "NONE", "BILINEAR","", "", "")
print ("Excess Green pyramids successfully calculated")

You have successfully exported the Excess Green.tif file
Excess Green pyramids successfully calculated


In [24]:
# Sharpening the image so the objects (crops & weeds) details are highlighted
sharpen_ExGreen = arcpy.ia.Convolution(ExGreen, 20)
sharpen_ExGreen.save(r'E:\Nitin.Rai\ASABE2021\Dataset\ExGreen_Sharpened4.tif')
print("You have successfully sharpened the Excess Green.tif file!")

You have successfully sharpened the Excess Green.tif file!


In [15]:
# env.outputCoordinateSystem = arcpy.SpatialReference("WGS_1984_UTM_Zone_14N")

In [26]:
binary_raster = arcpy.ia.Threshold(RGB_imagery)
binary_raster.save(r"E:\Nitin.Rai\ASABE2021\Dataset\ThresholdSharpen4.tif")
print("You have successfully perfromed Imagery Thresholding!")

You have successfully perfromed Imagery Thresholding!


In [12]:
# Data Preparation
dataset_path = r"C:\Users\Nitin.Rai\Desktop\ImageChipsPASCAL"
class_mapping = None
chip_size = 448
val_split = 0.3
batch_size = 64
transform = False
dataset_type = 'KITTI_rectangles'
prepare = prepare_data(dataset_path, class_mapping, chip_size, val_split, batch_size, transform,
                                        dataset_type)

You can deactivate this warning by passing `no_check=True`.


In [4]:
# # crop detection using YOLOv3
# in_folder = r'E:\Nitin.Rai\ASABE2021_New\ImageChipsPASCAL'
# out_model = r'E:\Nitin.Rai\ASABE2021_New\Model_Output\FasterRCNN'
# epochs = 100
# objectDetection_model = "FASTERRCNN"
# batch_size = 8
# arg = None
# lr = 0.001
# backbone_model = "RESNET50"
# pretrained_model = None
# validate = 30
# stop_training = "STOP_TRAINING"
# freeze = "UNFREEZE_MODEL"
# TrainDeepLearningModel(in_folder, out_model, epochs, objectDetection_model, batch_size, arg, lr, backbone_model,
#                       pretrained_model, validate, stop_training, freeze)

In [4]:
# Training again using the previously acquired trained model on crop images
start_time = datetime.now()
dataset = r'E:\Nitin.Rai\ASABE2021\Dataset\ImageChips'
output_location = r'E:\Nitin.Rai\SunflowerCropDetection\ModelOutput2\RetinaNetReTrained'
eph = 50
objectDetection_model = "RETINANET"
batch_size = 64
arg = "SCALES '[1,1,0.8]'; RATIOS '[0.5,1,1.5]'; chip_size:416"
lr = 0.001
backbone_model = "RESNET50"
pretrained_model = None
validate = 20
stop_training = "STOP_TRAINING"
freeze = "UNFREEZE_MODEL"
TrainDeepLearningModel(dataset, output_location, eph, objectDetection_model, batch_size, arg, lr, backbone_model,
                       pretrained_model, validate, stop_training, freeze)
end_time = datetime.now()
print("Duration of Training the dataset was: {}" .format(end_time - start_time))

epoch,train_loss,valid_loss,time
0,0.497636,9.015349,1:11:39
1,0.336148,3.302514,1:14:48
2,0.302375,1.744264,1:09:13
3,0.271287,0.657658,1:09:35
4,0.275251,0.456048,1:09:34
5,0.271608,3.131080,1:09:35
6,0.284302,1.207315,1:10:03
7,0.267650,1.587068,1:09:15
8,0.311753,1.766675,1:09:11
9,0.343694,1.272114,1:08:49


Epoch 10: early stopping


Computing model metrics...


Duration of Training the dataset was: 13:07:32.798546


In [7]:
# # Training the data with FasterRCNN
# data = data_prepare
# backbone = 'resnet50'
# pretrained_path = r"E:\Nitin.Rai\ASABE2021\Model Outputs\FasterRCNN"
# FasterRCNN(data, backbone, pretrained_path)

In [6]:
# data_prepare.show_batch()

In [11]:
# print("ArcGIS Online Org account")    
# gis = GIS("https://www.arcgis.com", "nitin.rai_NDUS", "jesusfreaks5")
# print("Logged in as " + str(gis.properties.user.username))

ArcGIS Online Org account
Logged in as nitin.rai_NDUS
